In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils.np_utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def generator(dir,
              gen=image.ImageDataGenerator(rescale=1./255),
              shuffle=True,
              batch_size=1,
              target_size=(24,24),
              class_mode='categorical' ):

    return gen.flow_from_directory(dir,
                                   batch_size=batch_size,
                                   shuffle=shuffle,
                                   color_mode='grayscale',
                                   class_mode=class_mode,
                                   target_size=target_size)

BS= 32
TS=(24,24)
train_batch= generator('a2a_data/train',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
valid_batch= generator('a2a_data/valid',
                       shuffle=True, 
                       batch_size=BS,
                       target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 5422 images belonging to 2 classes.
Found 1160 images belonging to 2 classes.
169 36


In [3]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)), 
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
   
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
from keras import backend as K

def precision(y_target, y_pred):
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)

    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    return precision


def recall(y_target, y_pred):
    y_target_yn = K.round(K.clip(y_target, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))

    count_true_positive = K.sum(y_target_yn * y_pred_yn) 
    count_true_positive_false_negative = K.sum(y_target_yn)

    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    return recall


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    return _f1score

In [5]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall, f1score])

model.fit_generator(train_batch, 
                    validation_data=valid_batch,
                    epochs=15,
                    steps_per_epoch=SPE,
                    validation_steps=VS)

model.save('a2a_model.h5', overwrite=True)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/15
169/169 [==============================] - 5s 31ms/step - loss: 0.5008 - acc: 0.7513 - precision: 0.7513 - recall: 0.7513 - f1score: 0.7513 - val_loss: 0.3137 - val_acc: 0.8819 - val_precision: 0.8819 - val_recall: 0.8819 - val_f1score: 0.8819
Epoch 2/15
169/169 [==============================] - 2s 11ms/step - loss: 0.3024 - acc: 0.8749 - precision: 0.8749 - recall: 0.8749 - f1score: 0.8749 - val_loss: 0.2333 - val_acc: 0.9078 - val_precision: 0.9078 - val_recall: 0.9078 - val_f1score: 0.9078
Epoch 3/15
169/169 [==============================] - 2s 11ms/step - loss: 0.1912 - acc: 0.9245 - precision: 0.9245 - recall: 0.9245 - f1score: 0.9245 - val_loss: 0.1445 - val_acc: 0.9486 - val_precision: 0.9486 - val_recall: 0.9486 - val_f1score: 0.9486
Epoch 4/15
169/169 [==============================] - 2s 11ms/step - loss: 0.1377 - acc: 0.9499 - precision: 0.9499 - recall: 0.9499 - f1sc

In [6]:
_loss, _acc, _precision, _recall, _f1score = model.evaluate_generator(valid_batch, steps=500)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))

loss: 0.148, accuracy: 0.954, precision: 0.954, recall: 0.954, f1score: 0.954
